In [1]:
import GPy
import pandas as pd 
import numpy as np 
import os
import sys 
sys.path.append('../')
from dispersant_screener.definitions import FEATURES
DATADIR = '../data'

from dispersant_screener.pal import pal
from dispersant_screener.definitions import FEATURES
from dispersant_screener.utils import get_maxmin_samples, get_kmeans_samples 
from dispersant_screener.gp import build_model, build_coregionalized_model

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, RationalQuadratic, Matern, RationalQuadratic, DotProduct
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold

In [2]:
df_full_factorial_feat = pd.read_csv(os.path.join(DATADIR, 'new_features_full_random.csv'))[FEATURES].values
a2 = pd.read_csv(os.path.join(DATADIR, 'b1-b21_random_virial_large_new.csv'))['A2_normalized'].values
deltaGMax = pd.read_csv(os.path.join(DATADIR, 'b1-b21_random_virial_large_new.csv'))['A2_normalized'].values
gibbs = pd.read_csv(os.path.join(DATADIR, 'b1-b21_random_deltaG.csv'))['deltaGmin'].values  * (-1)
gibbs_max = pd.read_csv(os.path.join(DATADIR, 'b1-b21_random_virial_large_new.csv'))['deltaGmax'].values
force_max =  pd.read_csv(os.path.join(DATADIR, 'b1-b21_random_virial_large_fit2.csv'))['F_repel_max'].values
rg = pd.read_csv(os.path.join(DATADIR, 'rg_results.csv'))['Rg'].values
y = np.hstack([rg.reshape(-1,1), gibbs.reshape(-1,1), gibbs_max.reshape(-1,1)])
assert len(df_full_factorial_feat) == len(a2) == len(gibbs) == len(y)

In [3]:
vt = VarianceThreshold(0.2)
X = vt.fit_transform(df_full_factorial_feat)

feat_scaler = StandardScaler()
X = feat_scaler.fit_transform(X)

# label_scaler = StandardScaler()
# y = label_scaler.fit_transform(y)

X_train, y_train, greedy_indices = get_kmeans_samples(X, y, 20)
X_train2, y_train2, greedy_indices2 = get_kmeans_samples(X, y, 21)


In [4]:
K = GPy.kern.RBF(X_train.shape[1])

In [5]:
icm = GPy.util.multioutput.ICM(input_dim=X_train.shape[1], num_outputs=3, kernel=K)

In [6]:
target_list = [y_train[:, i].reshape(-1, 1) for i in range(3)]
target_list2 = [y_train2[:, i].reshape(-1, 1) for i in range(3)]

In [7]:
m = GPy.models.GPCoregionalizedRegression([X_train] * 3, target_list, kernel=icm)

In [37]:
len(m.output_index)

60

In [38]:
m.set_XY([X_train2] * 3, target_list2)

In [39]:
m

gp.,value,constraints,priors
ICM.rbf.variance,1.0,+ve,
ICM.rbf.lengthscale,1.0,+ve,
ICM.B.W,"(3, 1)",,
ICM.B.kappa,"(3,)",+ve,
mixed_noise.Gaussian_noise_0.variance,1.0,+ve,
mixed_noise.Gaussian_noise_1.variance,1.0,+ve,
mixed_noise.Gaussian_noise_2.variance,1.0,+ve,


In [40]:
len(m.output_index)

63

In [41]:
m['.*ICM.*.variance'].constrain_fixed(1.)

array([0])

In [57]:
m

gp.,value,constraints,priors
ICM.rbf.variance,1.0,+ve fixed,
ICM.rbf.lengthscale,1.0,+ve,
ICM.B.W,"(3, 1)",,
ICM.B.kappa,"(3,)",+ve,
mixed_noise.Gaussian_noise_0.variance,1.0,+ve,
mixed_noise.Gaussian_noise_1.variance,1.0,+ve,
mixed_noise.Gaussian_noise_2.variance,1.0,+ve,


In [19]:
m.optimize()

In [42]:
m = build_coregionalized_model(X_train, y_train)

In [43]:
m

gp.,value,constraints,priors
ICM.Mat32.variance,1.0,+ve fixed,
ICM.Mat32.lengthscale,1.0,+ve,
ICM.B.W,"(3, 1)",,
ICM.B.kappa,"(3,)",+ve,
mixed_noise.Gaussian_noise_0.variance,1.0,+ve,
mixed_noise.Gaussian_noise_1.variance,1.0,+ve,
mixed_noise.Gaussian_noise_2.variance,1.0,+ve,


In [44]:
m.optimize()

In [45]:
m.set_XY([X_train2]*3, target_list2)

In [46]:
m

gp.,value,constraints,priors
ICM.Mat32.variance,1.0,+ve fixed,
ICM.Mat32.lengthscale,34.75076115315868,+ve,
ICM.B.W,"(3, 1)",,
ICM.B.kappa,"(3,)",+ve,
mixed_noise.Gaussian_noise_0.variance,3.131590022192877e-13,+ve,
mixed_noise.Gaussian_noise_1.variance,1.3225750406173606,+ve,
mixed_noise.Gaussian_noise_2.variance,0.11582181592988659,+ve,


In [47]:
m.optimize()